In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import json

# Backend

In [78]:
# import circuits data
circuit_df = pd.read_json("gp_2023.json")
circuit_df["lap length"] = circuit_df["lap nb"] * circuit_df["length"]

# import gp data
gp_df = pd.read_json("data_2023.json")

# import lap data
lap_df = pd.read_json("lap_data.json")

# colors
colors = json.loads(open("colors.json").read())

In [58]:
# gp data backend

# Import data
data_all = pd.read_json('data_2023.json')
data_all["index"] = data_all.groupby("pilot").cumcount()
data_all["index"] += 1

# dataset of the cumulative points of each pilot
data_points = data_all[["pilot", "points"]].copy()
data_points["cum_points"] = data_points.groupby("pilot").cumsum()
data_points["index"] = data_points.groupby("pilot").cumcount()

# extraction of pilots per team
with open("teams_2023.json") as jsonfile:
    teams_pilot = json.load(jsonfile)

data_team_points = {"team": [], "points": [], "cum_points": [], "index": []}
for i in data_points["index"].unique():
    for team in teams_pilot:
        data_team_points["team"].append(team)
        data_team_points["points"].append(data_points[data_points["index"] == i][data_points["pilot"].isin(teams_pilot[team])]["points"].sum())
        data_team_points["cum_points"].append(data_points[data_points["index"] == i][data_points["pilot"].isin(teams_pilot[team])]["cum_points"].sum())
        data_team_points["index"].append(i)
data_team_points = pd.DataFrame(data_team_points)

# extract the battles between pilots
with open("battles.json") as jsonfile:
    battles = json.load(jsonfile)
pilot_battles = battles["battles"]["pilots"]

df_battles_pilot = []  # list of battles dataframes
for battle in pilot_battles:
    # filter data with pilots of the battle
    pilots = list(battle.values())[0]
    df_bt = data_points[data_points["pilot"].isin(pilots)]
    
    # # compute the difference of points
    base_pilot = list(battle.keys())[0]
    df_bt["diff"] = df_bt["cum_points"] - df_bt[df_bt["pilot"] == base_pilot]["cum_points"].values[df_bt["index"]]
    df_battles_pilot.append(df_bt)

# extract the battles between teams
with open("battles.json") as jsonfile:
    battles = json.load(jsonfile)
team_battles = battles["battles"]["teams"]

df_battles_team = []  # list of battles dataframes
for battle in team_battles:
    # filter data with teams of the battle
    teams = list(battle.values())[0]
    df_bt = data_team_points[data_team_points["team"].isin(teams)]
    
    # # compute the difference of points
    base_team = list(battle.keys())[0]
    df_bt["diff"] = df_bt["cum_points"] - df_bt[df_bt["team"] == base_team]["cum_points"].values[df_bt["index"]]
    df_battles_team.append(df_bt)

C:\Users\shayk\AppData\Local\Temp\ipykernel_23628\2028418015.py:21: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_23628\2028418015.py:22: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_23628\2028418015.py:21: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_23628\2028418015.py:22: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_23628\2028418015.py:21: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_23628\2028418015.py:22: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_23628\2028418015.py:21: UserWarning:

Boolean Series key will be reindexed to ma

# Frontend

## GP data

### battles

In [12]:
px.line(df_battles_pilot[0], x="index", y="diff", color="pilot", title="Pilot battle", template="plotly_dark")

In [11]:
px.line(df_battles_pilot[1], x="index", y="diff", color="pilot", title="Pilot battle", template="plotly_dark")

In [84]:
px.line(df_battles_team[0], x="index", y="diff", color="team", color_discrete_map=colors["team"], title="Team battle", template="plotly_dark")

In [14]:
px.line(df_battles_team[1], x="index", y="diff", color="team", title="Team battle", template="plotly_dark")

In [15]:
px.line(df_battles_team[2], x="index", y="diff", color="team", title="Team battle", template="plotly_dark")

## Circuit data

In [17]:
fig = go.Figure(layout=go.Layout(title="length", template="plotly_dark"))
fig.add_trace(go.Box(y=circuit_df["length"], name="boxplot circuit length", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=circuit_df["name"]))

In [19]:
fig = go.Figure(layout=go.Layout(title="length", template="plotly_dark"))
fig.add_traces(go.Box(y=circuit_df["lap nb"], name="boxplot number of lap", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=circuit_df["name"]))

In [21]:
px.scatter_geo(circuit_df, lat="lat", lon="long", hover_name="name", title="circuit location", template="plotly_dark")

In [28]:
px.scatter(circuit_df, x="lap nb", y="length", hover_name="name", title="circuit length vs lap nb (lap_nb * length ~= 300 000", template="plotly_dark", hover_data=["lap length"])

In [63]:
fig = go.Figure(layout=go.Layout(title="pilot result", template="plotly_dark"))
fig.add_traces(go.Box(x=gp_df["pilot"], y=gp_df["result"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

In [62]:
fig = go.Figure(layout=go.Layout(title="circuit average speed", template="plotly_dark"))
fig.add_traces(go.Box(x=gp_df["circuit_name"], y=gp_df["avg_speed"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

In [61]:
# imshow du result avec le pilot en x et le circuit en y
px.imshow(gp_df.pivot("circuit_name", "pilot", "result"), title="result", template="plotly_dark")

In [60]:
fig = px.line(gp_df, x="circuit_name", y="result", color="pilot", title="result", markers=True, template="plotly_dark")
fig.update_layout(yaxis={'autorange': 'reversed'})

In [57]:
# print the row of the gp_df data with hul has pilot


   pilot  team  avg_speed  best_lap  grid  points  result      time  \
19   hul  haas        NaN       NaN   NaN       0     NaN       NaN   
39   hul  haas    198.198       NaN  20.0       0     NaN  5853.584   

   circuit_name  
19       Monaco  
39    Sao Paulo  


## Laps

In [68]:
fig = go.Figure(layout=go.Layout(title="lap time along gp", template="plotly_dark"))
fig.add_traces(go.Box(x=lap_df["lap"], y=lap_df["time"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

In [69]:
fig = go.Figure(layout=go.Layout(title="lap time along gp", template="plotly_dark"))
fig.add_traces(go.Box(x=lap_df["pilot"], y=lap_df["time"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

In [70]:
px.line(lap_df, x="lap", y="time", color="pilot", title="lap time along gp", template="plotly_dark")

In [71]:
px.imshow(lap_df.pivot("lap", "pilot", "time"), title="lap time along gp", template="plotly_dark")

In [76]:
fig = px.line(lap_df, x="lap", y="position", color="pilot", title="lap position along gp", markers=True, template="plotly_dark")
fig.update_layout(yaxis={'autorange': 'reversed'})

## Colors

In [83]:
px.line(lap_df, x="lap", y="time", color="pilot", color_discrete_map=colors["pilot"], title="lap speed along gp", markers=True, template="plotly_dark")